In [11]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np 
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
# Перевести MNIST фото файлы в 4х мерный вектор? (число изображений, высота, ширина, цветовой канал) 
transform = transforms.ToTensor()

In [19]:
# Тренировочные данные
train_data = datasets.MNIST(root='\cnn_data',train=True, 
                            download=True, transform=transform)

In [20]:
# Тестовые данные
test_data = datasets.MNIST(root='\cnn_data',train=False, 
                            download=True, transform=transform)

In [21]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: \cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [22]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: \cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()